In [1]:
import os
os.chdir('../../')

In [2]:
import pandas as pd
import pandas_gbq
from google.oauth2 import service_account

In [3]:
SERVICE_ACCOUNT = 'private/bigquery-write-access.json'
DATASET = 'student_counts'
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT)

# USA ennrollments

In [7]:
df = pd.read_csv('data/USA_enrollments.csv', index_col=['Country', 'Year'], encoding="utf-8")
df.columns.name = 'study_level'
df = df.stack()
df.name = 'students_count'
df = df.reset_index()

In [5]:
df.to_csv('data/USA_enrollments_transformed.csv', index=False)

# UK ennrollments

In [12]:
study_level_map = {"UG": "UG", "UG Other": "UG", "PG R": "PG", "PG CW": "PG", "PG Other": "PG"}

In [13]:
df = pd.read_csv('data/UK_students_count.csv', index_col=['hesa_country', 'country', 'year'],
                 encoding="utf-8").fillna(0).astype(int)
df.columns.name = 'detailed_study_level'
df = df.stack()
df.name = 'students_count'
df = df.reset_index()
df['study_level'] = df.detailed_study_level.apply(lambda x: study_level_map[x])
df['country'] = df.country.str.strip()
df.head()

,hesa_country,country,year,detailed_study_level,students_count,study_level
0,Albania,Albania,2004/2005,PG R,30,PG
1,Albania,Albania,2004/2005,PG CW,55,PG
2,Albania,Albania,2004/2005,PG Other,15,PG
3,Albania,Albania,2004/2005,UG,115,UG
4,Albania,Albania,2004/2005,UG Other,20,UG


In [14]:
table_full_name = '{}.{}'.format(DATASET, 'uk_int_student_counts')
df.to_gbq(table_full_name, credentials.project_id, credentials=credentials, if_exists='replace')

1it [00:15, 15.36s/it]


# Canada

In [15]:
df = pd.read_csv('data/Canada_international_students.csv', encoding="utf-8")
df['students_count'] = df.students_count.fillna(0).astype(int)
df['country'] = df.country.str.strip()
df.head()

,year,institution_type,country,students_count
0,2000/2001,University,Greenland,0
1,2001/2002,University,Greenland,3
2,2002/2003,University,Greenland,0
3,2003/2004,University,Greenland,0
4,2004/2005,University,Greenland,0


In [16]:
table_full_name = '{}.{}'.format(DATASET, 'canada_int_student_counts')
df.to_gbq(table_full_name, credentials.project_id, credentials=credentials, if_exists='replace')

1it [00:14, 14.11s/it]
